import libraries(I provide all libs that I need when make this tasks, if you need some external import them here)

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col
from pyspark.sql.functions import max, avg, min, desc
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql.functions import when

create local SparkSession

In [2]:
spark = SparkSession.builder \
    .appName("LocalSparkSession") \
    .master("local[*]") \
    .getOrCreate()
#для проверки количества потоков 
print("Default parallelism:", spark.sparkContext.defaultParallelism) 
#.master("spark://master-{node_ip}:7077") - если хотим использовать ресурсы всех машин в кластере 
#.master("yarn") - ресурсы выделяются через YARN ResourceManager.
#.master("k8s://https://<k8s-api-server>:6443") в кубернетис 

#Пример кода для подключения к кластеру
'''spark = SparkSession.builder \
    .appName("ClusterApp") \
    .master("spark://192.168.1.100:7077") - адрес Spark Master  \
    .config("spark.executor.memory", "4g") - сколько памяти выделить на каждого executor\
    .config("spark.executor.cores", "2") - сколько CPU-ядер у каждого executor\
    .getOrCreate()'''



Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/09/01 13:58:35 WARN Utils: Your hostname, MacBook-Air-Ulyana.local, resolves to a loopback address: 127.0.0.1; using 10.202.39.95 instead (on interface en0)
25/09/01 13:58:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/01 13:58:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Default parallelism: 8


'spark = SparkSession.builder     .appName("ClusterApp")     .master("spark://192.168.1.100:7077") - адрес Spark Master      .config("spark.executor.memory", "4g") - сколько памяти выделить на каждого executor    .config("spark.executor.cores", "2") - сколько CPU-ядер у каждого executor    .getOrCreate()'

read csv with inferschema

In [3]:

df = spark.read.csv(
    "./ds_salaries.csv",          # путь к файлу
    header=True,         # первая строка - заголовки
    inferSchema=True     # автоопределение типов колонок
)

df.show(5)

+---+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|_c0|work_year|experience_level|employment_type|           job_title|salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|  0|     2020|              MI|             FT|      Data Scientist| 70000|            EUR|        79833|                DE|           0|              DE|           L|
|  1|     2020|              SE|             FT|Machine Learning ...|260000|            USD|       260000|                JP|           0|              JP|           S|
|  2|     2020|              SE|             FT|   Big Data Engineer| 85000|            GBP|       109024|                GB|          50|              GB|

read csv one more time with the same code and you will see that it almostly don't take time, because info already in SparkSession and it will not read nothing
from this file

In [4]:

df2 = spark.read.csv("./ds_salaries.csv", header=True, inferSchema=True)

write schema of scv on screen

In [5]:
df.printSchema()  

root
 |-- _c0: integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



create schema of this scv

In [6]:
schema=StructType([StructField('_c0', IntegerType(), True),
                   StructField('work_year', IntegerType(), True),
                   StructField('experience_level', StringType(), True),
                   StructField('employment_type', StringType(), True),
                   StructField('job_title', StringType(), True),
                   StructField('salary', IntegerType(), True),
                   StructField('salary_currency', StringType(), True),
                   StructField('salary_in_usd', IntegerType(), True),
                   StructField('employee_residence', StringType(), True),
                   StructField('remote_ratio', IntegerType(), True),
                   StructField('company_location', StringType(), True),
                   StructField('company_size', StringType(), True)
                   ])


restart kernel without cleaning output and after restarting you need to initialize SparkSession, after initialize start execute only cells from cell with schema=
=StructType.... 
To restart kernel click Kernel, Restart.

read ds_salaries with predefined schema and compare results from this cell and cell with inferSchema

In [7]:

spark2 = SparkSession.builder.appName('manual_read').master('local[*]').getOrCreate()
df3 = spark2.read.csv("./ds_salaries.csv", header=True,  schema=schema)



25/09/01 13:58:40 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


this happens because read operation is lazy(transformation), but if you use inferschema it start to be action that will create Spark Job, because Spark need to loop throw all file to check datatypes for all columns and this can harm to your code(if we compare to parquet, it will also go to check data types, but parquet provide meta information, so Spark will not go throw all file, he will just read meta information, but csv don't provide such meta information). Also header make Spark to create one more Spark Job to check first line
to define name of columns and remember to skeep it when reading. Actual reading start when you will use first action. More about Spark Jobs you will see in next topic

write schema of scv on screen one more time and compare with previous

In [8]:
df3.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



now continue to work with one of the dataframes that you create

print data in dataframe using df.show

In [9]:
df3.show(5)

+---+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|_c0|work_year|experience_level|employment_type|           job_title|salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|  0|     2020|              MI|             FT|      Data Scientist| 70000|            EUR|        79833|                DE|           0|              DE|           L|
|  1|     2020|              SE|             FT|Machine Learning ...|260000|            USD|       260000|                JP|           0|              JP|           S|
|  2|     2020|              SE|             FT|   Big Data Engineer| 85000|            GBP|       109024|                GB|          50|              GB|

print data in dataframe using display(df.toPandas())

In [10]:
display(df.toPandas())

,_c0,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L
...,...,...,...,...,...,...,...,...,...,...,...,...
602,602,2022,SE,FT,Data Engineer,154000,USD,154000,US,100,US,M
603,603,2022,SE,FT,Data Engineer,126000,USD,126000,US,100,US,M
604,604,2022,SE,FT,Data Analyst,129000,USD,129000,US,0,US,M
605,605,2022,SE,FT,Data Analyst,150000,USD,150000,US,100,US,M


create df_job_title that consists from all job_titles without duplicates

In [11]:
df_job_title = df.select('job_title').drop_duplicates()

print all rows from df_job_titles without truncating jobs

In [12]:
df_job_title.show(truncate=False)

+----------------------------------------+
|job_title                               |
+----------------------------------------+
|3D Computer Vision Researcher           |
|Lead Data Engineer                      |
|Head of Machine Learning                |
|Data Specialist                         |
|Data Analytics Lead                     |
|Machine Learning Scientist              |
|Lead Data Analyst                       |
|Data Engineering Manager                |
|Staff Data Scientist                    |
|ETL Developer                           |
|Director of Data Engineering            |
|Product Data Analyst                    |
|Principal Data Scientist                |
|AI Scientist                            |
|Director of Data Science                |
|Machine Learning Engineer               |
|Lead Data Scientist                     |
|Machine Learning Infrastructure Engineer|
|Data Science Engineer                   |
|Machine Learning Manager                |
+----------

create  df_analytic that will consists from max, avg, min USD salaries for all job_titles using groupBy. name of fields is avg_salary, min_salary, max_salary

In [13]:
df_analytic = df.groupBy("job_title")\
                                    .agg(
                                          avg("salary_in_usd").alias("avg_salary"),
                                          min("salary_in_usd").alias("min_salary"),
                                          max("salary_in_usd").alias("max_salary"))


print all rows from df_analytic without trancating jobs

In [14]:
df_analytic.show(truncate=False)

+----------------------------------------+------------------+----------+----------+
|job_title                               |avg_salary        |min_salary|max_salary|
+----------------------------------------+------------------+----------+----------+
|3D Computer Vision Researcher           |5409.0            |5409      |5409      |
|Lead Data Engineer                      |139724.5          |56000     |276000    |
|Head of Machine Learning                |79039.0           |79039     |79039     |
|Data Specialist                         |165000.0          |165000    |165000    |
|Data Analytics Lead                     |405000.0          |405000    |405000    |
|Machine Learning Scientist              |158412.5          |12000     |260000    |
|Lead Data Analyst                       |92203.0           |19609     |170000    |
|Data Engineering Manager                |123227.2          |59303     |174000    |
|Staff Data Scientist                    |105000.0          |105000    |1050

now you need to add in df_analytic column row_id, that will show order of all job_titles depending on avg salary. they should be descending

In [15]:
window = Window.orderBy(desc("avg_salary"))
df_analytic = df_analytic.withColumn("row_id", row_number().over(window))

print all data from df_analytic

In [16]:
df_analytic.show(truncate=False)

+----------------------------------+------------------+----------+----------+------+
|job_title                         |avg_salary        |min_salary|max_salary|row_id|
+----------------------------------+------------------+----------+----------+------+
|Data Analytics Lead               |405000.0          |405000    |405000    |1     |
|Principal Data Engineer           |328333.3333333333 |185000    |600000    |2     |
|Financial Data Analyst            |275000.0          |100000    |450000    |3     |
|Principal Data Scientist          |215242.42857142858|148261    |416000    |4     |
|Director of Data Science          |195074.0          |130026    |325000    |5     |
|Data Architect                    |177873.9090909091 |90700     |266400    |6     |
|Applied Data Scientist            |175655.0          |54238     |380000    |7     |
|Analytics Engineer                |175000.0          |135000    |205300    |8     |
|Data Specialist                   |165000.0          |165000    

25/09/01 13:58:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/09/01 13:58:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/09/01 13:58:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/09/01 13:58:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/09/01 13:58:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/09/01 13:58:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


it isn't beautifull, so we need to put now row_id on first place in df_analytic

In [17]:
df_analytic = df_analytic.select("row_id", *[c for c in df_analytic.columns if c != "row_id"])


print df_analytic now

In [18]:

df_analytic.show(10, truncate=False)

25/09/01 13:58:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/09/01 13:58:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/09/01 13:58:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+------+------------------------+------------------+----------+----------+
|row_id|job_title               |avg_salary        |min_salary|max_salary|
+------+------------------------+------------------+----------+----------+
|1     |Data Analytics Lead     |405000.0          |405000    |405000    |
|2     |Principal Data Engineer |328333.3333333333 |185000    |600000    |
|3     |Financial Data Analyst  |275000.0          |100000    |450000    |
|4     |Principal Data Scientist|215242.42857142858|148261    |416000    |
|5     |Director of Data Science|195074.0          |130026    |325000    |
|6     |Data Architect          |177873.9090909091 |90700     |266400    |
|7     |Applied Data Scientist  |175655.0          |54238     |380000    |
|8     |Analytics Engineer      |175000.0          |135000    |205300    |
|9     |Data Specialist         |165000.0          |165000    |165000    |
|10    |Head of Data            |160162.6          |32974     |235000    |
+------+-----------------

25/09/01 13:58:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/09/01 13:58:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/09/01 13:58:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


here you need to create df_exp_lvl with the biggest usd_salary(biggest_salary) for each experience_level(you need to save all fields like in entire dataframe)

In [19]:
window_spec = Window.partitionBy('experience_level').orderBy(col('salary_in_usd').desc())
df_with_rank = df.withColumn('rank', row_number().over(window_spec))
df_exp_lvl = df_with_rank.filter(col('rank') == 1).drop('rank')


print here df_exp_lvl

In [20]:
df_exp_lvl.show(truncate=False)

+---+---------+----------------+---------------+-------------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|_c0|work_year|experience_level|employment_type|job_title                |salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---+---------+----------------+---------------+-------------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|37 |2020     |EN              |FT             |Machine Learning Engineer|250000|USD            |250000       |US                |50          |US              |L           |
|252|2021     |EX              |FT             |Principal Data Engineer  |600000|USD            |600000       |US                |100         |US              |L           |
|33 |2020     |MI              |FT             |Research Scientist       |450000|USD            |450000       |US                |

create df_best that consists from rows where salary of guy same as biggest salary for other people in his exp_lvl and choose only columns: id, experience_level, biggest_salary, employee_residence

In [21]:

window_spec = Window.partitionBy('experience_level')

df_with_max = df.withColumn('biggest_salary', max('salary_in_usd').over(window_spec))
df_best = df_with_max.filter(col('salary_in_usd') == col('biggest_salary')).select('_c0', 'experience_level', 'biggest_salary', 'employee_residence')




print df_best

In [22]:
df_best.show(truncate=False)

+---+----------------+--------------+------------------+
|_c0|experience_level|biggest_salary|employee_residence|
+---+----------------+--------------+------------------+
|37 |EN              |250000        |US                |
|252|EX              |600000        |US                |
|33 |MI              |450000        |US                |
|97 |MI              |450000        |US                |
|63 |SE              |412000        |US                |
+---+----------------+--------------+------------------+



drop duplicates if exist by experience_level

In [23]:
df_best = df_best.dropDuplicates(['experience_level'])


print df_best

In [24]:

df_best.show(truncate=False)

+---+----------------+--------------+------------------+
|_c0|experience_level|biggest_salary|employee_residence|
+---+----------------+--------------+------------------+
|37 |EN              |250000        |US                |
|252|EX              |600000        |US                |
|33 |MI              |450000        |US                |
|63 |SE              |412000        |US                |
+---+----------------+--------------+------------------+



create df_new_best from df_best without id, and make the next: when exp_level = MI we want middle, when SE we want senior, else Null

In [25]:
df_new_best = df_best.drop('id').withColumn(
    'experience_level',
    when(col('experience_level') == 'MI', 'middle')
    .when(col('experience_level') == 'SE', 'senior')
    .otherwise(None)
)


print df_new_best

In [26]:
df_new_best.show()

+---+----------------+--------------+------------------+
|_c0|experience_level|biggest_salary|employee_residence|
+---+----------------+--------------+------------------+
| 37|            NULL|        250000|                US|
|252|            NULL|        600000|                US|
| 33|          middle|        450000|                US|
| 63|          senior|        412000|                US|
+---+----------------+--------------+------------------+



write df_new_best like 1.csv and load then it to df_final

In [27]:
df_new_best.write.mode("overwrite").option("header", True).csv("1.csv")
df_final = spark.read.csv("1.csv", header=True, inferSchema=True)

print df_final

In [28]:
df_final.show() 

+---+----------------+--------------+------------------+
|_c0|experience_level|biggest_salary|employee_residence|
+---+----------------+--------------+------------------+
| 37|            NULL|        250000|                US|
|252|            NULL|        600000|                US|
| 33|          middle|        450000|                US|
| 63|          senior|        412000|                US|
+---+----------------+--------------+------------------+



filter df_final to delete experience_level where it Null, then join this table by biggest_salary(salary_in_usd) and employee_residence with entire df

In [29]:
df.final = df_final.filter("experience_level IS NOT NULL") 
df_final = df_final.join(df, df_final.biggest_salary==df.salary_in_usd)

print df_final

In [30]:
df_final.show()

+---+----------------+--------------+------------------+---+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|_c0|experience_level|biggest_salary|employee_residence|_c0|work_year|experience_level|employment_type|           job_title|salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---+----------------+--------------+------------------+---+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
| 33|          middle|        450000|                US| 33|     2020|              MI|             FT|  Research Scientist|450000|            USD|       450000|                US|           0|              US|           M|
| 37|            NULL|        250000|                US| 37|     2020|              EN|             FT|M

last task is to save in variable and then print this variable of the biggest salary_in_usd from df_final

In [31]:

variable = df_final.agg(max('salary_in_usd').alias('max_salary')).collect()[0]['max_salary']
print(int(variable))


600000


It is the end of PySpark basics. In other lessons you will learn optimizations technics and how to make distributed system